### Imports

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import math

### Set up Mediapipe

Initialize Mediapipe pose

In [3]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

Calculating angle of Joints

In [4]:
# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Second point (center point)
    c = np.array(c)  # Third point

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

Initialize path to the video data, and variable to store the output

In [5]:
# Path to the training video
video_path = "path_to_your_video.mp4"
cap = cv2.VideoCapture(video_path)

# Array to store joint angles for each frame
exercise_data = []

Start Pose capture with Computer Vision

In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Make detection
    results = pose.process(image)

    # Extract landmarks if detected
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # Extract joint coordinates (example for jumping jacks: shoulders, hips, knees)
        left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
        left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
        right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
        left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
        right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]

        # Calculate angles for jumping jacks (shoulder-hip-knee angles)
        left_shoulder_angle = calculate_angle(left_hip, left_shoulder, right_shoulder)
        right_shoulder_angle = calculate_angle(right_hip, right_shoulder, left_shoulder)
        left_hip_angle = calculate_angle(left_knee, left_hip, left_shoulder)
        right_hip_angle = calculate_angle(right_knee, right_hip, right_shoulder)

        # Store angles for the current frame
        frame_data = [left_shoulder_angle, right_shoulder_angle, left_hip_angle, right_hip_angle]
        exercise_data.append(frame_data)

    # Draw pose landmarks for visualization (optional)
    mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Display the frame (optional)
    cv2.imshow('Pose Tracking', frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()